In [1]:
import google.cloud.bigquery as bq
import pandas_gbq
import warnings
warnings.filterwarnings('ignore')
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly_express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots


from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import RidgeClassifier

import itertools
import collections
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from scipy.stats import norm
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
%matplotlib inline


In [2]:
# pip install plotly_express

In [3]:
project_id = '.................'
df = pd.read_gbq('select * from ad_hoc.effective_call_list',project_id = project_id)

In [4]:
df1 = df.copy()

In [5]:
df1

,store_id,call_date,store_category,store_region,credit_balance,live_ads,avg_price_per_live_ad,credit_purchased,credit_expired,credit_listing,credit_ps,converted
0,1053937,2021-03-31,2040,1,0,9,5.991000e+05,100,0,63.999999972,28.000000028,1
1,1050779,2021-02-28,2040,1,25,26,6.258581e+06,105,0,80,0,1
2,1048340,2021-02-28,2040,1,0,81,3.159933e+05,420,0,405,0,0
3,1000368,2021-03-19,1060,2,0,125,5.833120e+02,575,0,695,0,1
4,151754,2021-03-30,2020,2,0,112,3.749057e+05,1150,0,675,855,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2041,1050983,2021-02-28,2020,16,0,0,0.000000e+00,0,0,0,0,0
2042,927883,2021-02-28,2040,16,354,219,8.644765e+05,4135,0,1343,2517,1
2043,1048354,2021-02-28,2020,16,0,48,5.731146e+05,730,0,190,0,1
2044,937190,2020-09-23,2100,17,0,0,0.000000e+00,0,392,0,0,0


In [6]:
X_rs = df1[['store_category', 'store_region', 'credit_balance', 'live_ads', 'avg_price_per_live_ad', 'credit_purchased', 'credit_expired', 'credit_listing', 'credit_ps']]
y_rs = df1['converted']

df1=df.copy()

In [7]:
print("Shape of y:", y_rs.shape)
print("Shape of x:", X_rs.shape)

Shape of y: (2046,)
Shape of x: (2046, 9)


In [8]:
# df_rs = pd.DataFrame(np.hstack((X_rs,y_rs[:, None])), columns=df2.columns)
# df_rs.converted = df_rs.converted.astype(int)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_rs, y_rs, test_size=0.2, random_state=42)

In [10]:
categories = X_rs['store_category'].unique()
regions = X_rs['store_region'].unique()

convert = ColumnTransformer(
    transformers = [('one_hot_encoder', OneHotEncoder(categories=[categories, regions], handle_unknown='ignore'), 
                ['store_category', 'store_region'])],
    remainder='passthrough')

transformed = Pipeline(steps=[('convert', convert),('scale', StandardScaler(with_mean=False))])
clf= transformed.fit_transform(X_rs)

In [11]:
classifiers = {
      "Logistic Regression": LogisticRegression(),
      # "Support Vector Classifier": SVC(),
      "Decision Tree": DecisionTreeClassifier(),
      "KNearest": KNeighborsClassifier(),
      "Random Forest": RandomForestClassifier(),
      "MLP": MLPClassifier(),
      "Ridge": RidgeClassifier(),
      "Stochastic Gradient Boosting": GradientBoostingClassifier()
      
  }

In [12]:
print('Cross-Validation Scores:-')
for key, classifier in classifiers.items():
    classifier.fit(X_train, y_train)
    cv_score = cross_val_score(classifier, X_train, y_train, cv=5) 
    #number of cv you have to try for each selected set of hyperparameters
    print('{}: {}'.format(key,round(cv_score.mean()*100.0, 2)))

Cross-Validation Scores:-
Logistic Regression: 62.29
Decision Tree: 68.77
KNearest: 60.45
Random Forest: 73.35
MLP: 56.17
Ridge: 65.77
Stochastic Gradient Boosting: 73.11


In [13]:
params = {
    "Logistic Regression"         : {"penalty": ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
    # "Support Vector Classifier"   : {'C': [0.5, 0.7, 0.9, 1], 'kernel': ['rbf', 'poly', 'sigmoid', 'linear']},
    "Decision Tree"               : {"criterion": ["gini", "entropy"], "max_depth": list(range(2,4,1)),"min_samples_leaf": list(range(5,7,1))},
    "KNearest"                    : {"n_neighbors": list(range(2,5,1)), 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']},
    "Random Forest"               : {'n_estimators': [10, 100, 1000],'max_features':['auto', 'sqrt']},
    "MLP"                         : {'solver': ['lbfgs'], 'max_iter': [1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000 ],'alpha': 10.0 ** -np.arange(1, 10),
                                     'hidden_layer_sizes':np.arange(10, 15), 'random_state':[0,1,2,3,4,5,6,7,8,9]},
    "Ridge"                       : {'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]},
    "Stochastic Gradient Boosting": {'n_estimators':range(20,81,10),'max_features':['sqrt']}
}

In [14]:
def gridsearch(classifier, params):
    grid_classifier = GridSearchCV(classifier, params)
    grid_classifier.fit(X_train, y_train)
    best_classifier = grid_classifier.best_estimator_
    return best_classifier

In [ ]:
print('Cross-Validation Scores after applying GridSearch -')
for key, classifier in classifiers.items():
    classifier = gridsearch(classifier,params[key])
    cv_score = cross_val_score(classifier, X_train, y_train, cv=5)
    print('{}: {}'.format(key,round(cv_score.mean()*100.0, 2)))

Cross-Validation Scores after applying GridSearch -
Logistic Regression: 62.41
Decision Tree: 67.91
KNearest: 60.02
Random Forest: 73.29
MLP: 60.58
Ridge: 65.77
Stochastic Gradient Boosting: 71.33


In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    ####Learning curve ####### To check if there's any overfitting
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(name='Training score - Standard Deviation',x=train_sizes,y=train_scores_mean+train_scores_std,
                            mode='lines',showlegend=False,marker=dict(color='blue')))
    
    fig.add_trace(go.Scatter(name='Training score', x=train_sizes, y=train_scores_mean, fill='tonexty',mode='lines+markers',
                            marker=dict(color='blue')))
    
    fig.add_trace(go.Scatter(name='Training score + Standard Deviation', x=train_sizes, y=train_scores_mean-train_scores_std,
                            mode='lines', fill='tonexty', showlegend=False,marker=dict(color='blue')))
    
    fig.add_trace(go.Scatter(x=train_sizes,y=test_scores_mean+test_scores_std,mode='lines',showlegend=False,
                             marker=dict(color='orange')))
   
    fig.add_trace(go.Scatter(name='Validation Score',x=train_sizes, y=test_scores_mean, mode='lines+markers',fill='tonexty',
                            marker=dict(color='orange')))

    fig.add_trace(go.Scatter(x=train_sizes,y=test_scores_mean-test_scores_std,mode='lines',fill='tonexty',showlegend=False,
                            marker=dict(color='orange')))

    fig.update_layout(width=700,height=400,template='seaborn',title=title,margin=dict(l=60,r=0,b=0,t=40),legend=dict(orientation='h',x=0.5,y=1),
                        xaxis=dict(title='Training examples',mirror=True,linecolor='black',linewidth=2),
                        yaxis=dict(title='Scores',range=ylim if ylim is not None else None,
                        mirror=True,linecolor='black',linewidth=2))
    return fig

In [ ]:
n_jobs = 10
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=42)
log_reg = gridsearch(LogisticRegression(), params['Logistic Regression'])
# svc = gridsearch(SVC(), params['Support Vector Classifier'])
decision_tree = gridsearch(DecisionTreeClassifier(), params['Decision Tree'])
knearest = gridsearch(KNeighborsClassifier(), params['KNearest'])
random_forest = gridsearch(RandomForestClassifier(), params['Random Forest'])
MLP = gridsearch(MLPClassifier(),params['MLP'])
Ridge = gridsearch(RidgeClassifier(), params['Ridge'])
Stochastic_gradient_boosting = gridsearch(GradientBoostingClassifier(), params['Stochastic Gradient Boosting'])

In [ ]:
plot_learning_curve(log_reg,'Logistic Regression', X_train, y_train, n_jobs=10)

In [ ]:
# plot_learning_curve(svc, 'Support Vector Classifier', X_train, y_train, n_jobs=10)

In [ ]:
plot_learning_curve(decision_tree, 'Decision Tree Classifier', X_train, y_train, n_jobs=10)

In [ ]:
plot_learning_curve(knearest, 'KNearest Classifier', X_train, y_train, n_jobs=10)

In [ ]:
plot_learning_curve(random_forest, 'Random Forest Classifier', X_train, y_train, n_jobs=10)

In [ ]:
plot_learning_curve(MLP, 'MLP Classifier', X_train, y_train, n_jobs=10)

In [ ]:
plot_learning_curve(Ridge, 'Ridge Classifier', X_train, y_train, n_jobs=10)

In [ ]:
plot_learning_curve(Stochastic_gradient_boosting, 'Stochastic Gradient Boosting Classifier', X_train, y_train, n_jobs=10)

In [ ]:
###### KNN AND DT IS CLOSER TO IDEAL CASE ( closer to value '1')

In [ ]:
log_pred = log_reg.predict(X_test)
# svc_pred = svc.predict(X_test)
tree_pred = decision_tree.predict(X_test)
knear_pred = knearest.predict(X_test)
rfnear_pred = random_forest.predict(X_test)
mlpnear_pred = MLP.predict(X_test)
Rnear_pred = Ridge.predict(X_test)
SGBnear_pred = Stochastic_gradient_boosting.predict(X_test)

log_fpr, log_tpr, log_threshold = roc_curve(y_test, log_pred)
# svc_fpr, svc_tpr, svc_threshold = roc_curve(y_test, svc_pred)
tree_fpr, tree_tpr, tree_threshold = roc_curve(y_test, tree_pred)
knear_fpr, knear_tpr, knear_threshold = roc_curve(y_test, knear_pred)
rfnear_fpr, rfnear_tpr, rfnear_threshold = roc_curve(y_test, rfnear_pred)
mlpnear_fpr, mlpnear_tpr, mlpnear_threshold = roc_curve(y_test, mlpnear_pred)
Rnear_fpr, Rnear_tpr, Rnear_threshold = roc_curve(y_test, Rnear_pred)
SGBnear_fpr, SGBnear_tpr, SGBnear_threshold = roc_curve(y_test, SGBnear_pred)

fig = go.Figure()
fig.add_trace(go.Scatter(name='Logistic Regression Classifier Score: {:.4f}'.format(roc_auc_score(y_test, log_pred)),x=log_fpr,y=log_tpr,mode='lines'))
# fig.add_trace(go.Scatter(name='Support Vector Classifier Score: {:.4f}'.format(roc_auc_score(y_test, svc_pred)),x=svc_fpr,y=svc_tpr,mode='lines'))
fig.add_trace(go.Scatter(name='Decision Tree Classifier Score: {:.4f}'.format(roc_auc_score(y_test, tree_pred)),x=tree_fpr,y=tree_tpr,mode='lines'))
fig.add_trace(go.Scatter(name='K-Nearest Neighbors Classifier Score: {:.4f}'.format(roc_auc_score(y_test, knear_pred)),x=knear_fpr,y=knear_tpr,mode='lines'))
fig.add_trace(go.Scatter(name='Random Forest Classifier Score: {:.4f}'.format(roc_auc_score(y_test, rfnear_pred)),x=rfnear_fpr,y=rfnear_tpr,mode='lines'))
fig.add_trace(go.Scatter(name='MLP Classifier Score: {:.4f}'.format(roc_auc_score(y_test, mlpnear_pred)),x=mlpnear_fpr,y=mlpnear_tpr,mode='lines'))
fig.add_trace(go.Scatter(name='Ridge Classifier Score: {:.4f}'.format(roc_auc_score(y_test, Rnear_pred)),x=Rnear_fpr,y=Rnear_tpr,mode='lines'))
fig.add_trace(go.Scatter(name='Stochastic Gradient Classifier Score: {:.4f}'.format(roc_auc_score(y_test, SGBnear_pred)),x=SGBnear_fpr,y=SGBnear_tpr,mode='lines'))

fig.add_trace(go.Scatter(name='AUC-ROC=0.5',x=[0,1],y=[0,1],line=dict(dash='dot'),showlegend=False))
fig.update_layout(width=1000,height=600,xaxis=dict(mirror=True,linewidth=2,linecolor='black'),yaxis=dict(mirror=True,linewidth=2,linecolor='black'),
            title='ROC Curve<br>(All Classifiers)',template='seaborn',legend=dict(x=0.46,y=0,traceorder="normal", font=dict(family="sans-serif",
            size=12,color="black"),bgcolor="Lightgray",bordercolor="Black",borderwidth=2),
            annotations=[dict(x=0.5,y=0.5,xref="x", yref="y",text="Minimum ROC Score of 50% <br>",showarrow=True,arrowhead=7,ax=40,ay=50)])
fig.show()

In [ ]:
# From the above curve it can be seen that DT has the highest ROC curve SO, 
# it is the best model according to this metric.

In [ ]:
labels = ['Non-Converted', 'Converted']
print('#'*125)
print('Logistic Regression:')
print(classification_report(y_test, log_pred, target_names=labels))
print('#'*125)
# print('Support Vector Classifier:')
# print(classification_report(y_test, svc_pred, target_names=labels))
# print('#'*125)
print('Decision Tree:')
print(classification_report(y_test, tree_pred, target_names=labels))
print('#'*125)
print('KNearest Neighbours:')
print(classification_report(y_test, knear_pred, target_names=labels))
print('#'*125)
print('Random Forests:')
print(classification_report(y_test, rfnear_pred, target_names=labels))
print('#'*125)
print('MLP:')
print(classification_report(y_test, mlpnear_pred, target_names=labels))
print('#'*125)
print('Ridge:')
print(classification_report(y_test, Rnear_pred, target_names=labels))
print('#'*125)
print('Stochastic Gradient Boosting:')
print(classification_report(y_test, SGBnear_pred, target_names=labels))

In [ ]:
print('-'*65)
print('Ridge Classifier')
print('-'*65)

print('ROC/AUC score:',roc_auc_score(y_test,Rnear_pred))
print('Accuracy score:',accuracy_score(y_test, Rnear_pred))
print('F1 score:',f1_score(y_test, Rnear_pred))
print('-'*65)

print('-'*65)
print('Gradient Boosting Classifier')
print('-'*65)

print('ROC/AUC score:',roc_auc_score(y_test,SGBnear_pred))
print('Accuracy score:',accuracy_score(y_test, SGBnear_pred))
print('F1 score:',f1_score(y_test, SGBnear_pred))
print('-'*65)

print('-'*65)
print('MLP Classifier')
print('-'*65)

print('ROC/AUC score:',roc_auc_score(y_test,mlpnear_pred))
print('Accuracy score:',accuracy_score(y_test, mlpnear_pred))
print('F1 score:',f1_score(y_test, mlpnear_pred))
print('-'*65)

print('-'*65)
print('Decision Tree Classifier')
print('-'*65)

print('ROC/AUC score:',roc_auc_score(y_test,tree_pred))
print('Accuracy score:',accuracy_score(y_test, tree_pred))
print('F1 score:',f1_score(y_test, tree_pred))
print('-'*65)

print('-'*65)
print('K-nearest Classifier')
print('-'*65)

print('ROC/AUC score:',roc_auc_score(y_test,knear_pred))
print('Accuracy score:',accuracy_score(y_test, knear_pred))
print('F1 score:',f1_score(y_test, knear_pred))
print('-'*65)

print('-'*65)
print('Random Forest Classifier')
print('-'*65)

print('ROC/AUC score:',roc_auc_score(y_test,rfnear_pred))
print('Accuracy score:',accuracy_score(y_test, rfnear_pred))
print('F1 score:',f1_score(y_test, rfnear_pred))
print('Recall:',recall_score(y_test, rfnear_pred))
print('Precision:',precision_score(y_test, rfnear_pred))

print('-'*65)

print('-'*65)
print('Logistic Regression Classifier')
print('-'*65)

print('ROC/AUC score:',roc_auc_score(y_test,log_pred))
print('Accuracy score:',accuracy_score(y_test, log_pred))
print('F1 score:',f1_score(y_test, log_pred))
print('-'*65)
